In [47]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
df=pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(df[['age','affordibility']], df['bought_insurance'], test_size=0.2)

In [50]:
len(X_train)

22

In [51]:
len(X_test)

6

In [52]:
X_train_scaled=X_train.copy()
X_train_scaled['age']=X_train_scaled['age']/100

X_test_scaled=X_test.copy()
X_test_scaled['age']=X_test_scaled['age']/100




In [53]:
X_test_scaled

,age,affordibility
6,0.55,0
25,0.54,1
1,0.25,0
7,0.60,0
2,0.47,1
24,0.50,1


In [54]:
model=keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_scaled,y_train,epochs=2000)

Epoch 1/2000
1/1 [==============================] - 2s 2s/step - loss: 0.7807 - accuracy: 0.4545
Epoch 2/2000
1/1 [==============================] - 0s 9ms/step - loss: 0.7802 - accuracy: 0.4545
Epoch 3/2000
1/1 [==============================] - 0s 8ms/step - loss: 0.7797 - accuracy: 0.4545
Epoch 4/2000
1/1 [==============================] - 0s 8ms/step - loss: 0.7791 - accuracy: 0.4545
Epoch 5/2000
1/1 [==============================] - 0s 10ms/step - loss: 0.7786 - accuracy: 0.4545
Epoch 6/2000
1/1 [==============================] - 0s 9ms/step - loss: 0.7781 - accuracy: 0.4545
Epoch 7/2000
1/1 [==============================] - 0s 7ms/step - loss: 0.7776 - accuracy: 0.4545
Epoch 8/2000
1/1 [==============================] - 0s 9ms/step - loss: 0.7771 - accuracy: 0.4545
Epoch 9/2000
1/1 [==============================] - 0s 8ms/step - loss: 0.7766 - accuracy: 0.4545
Epoch 10/2000
1/1 [==============================] - 0s 7ms/step - loss: 0.7761 - accuracy: 0.4545
Epoch 11/2000
1/1 [

In [55]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 451ms/step - loss: 0.5417 - accuracy: 1.0000


[0.541681706905365, 1.0]

In [56]:
model.predict(X_test_scaled)

array([[0.4790391 ],
       [0.6228753 ],
       [0.33377415],
       [0.5043297 ],
       [0.5890583 ],
       [0.6036748 ]], dtype=float32)

In [57]:
X_test_scaled

,age,affordibility
6,0.55,0
25,0.54,1
1,0.25,0
7,0.60,0
2,0.47,1
24,0.50,1


In [58]:
y_test

6     0
25    1
1     0
7     1
2     1
24    1
Name: bought_insurance, dtype: int64

In [59]:
coef, intercept=model.get_weights()
coef, intercept

(array([[2.024238 ],
        [0.6059055]], dtype=float32),
 array([-1.1972238], dtype=float32))

In [60]:
def sigmoid(x):
    import math
    return 1/(1 + math.exp(-x))
sigmoid(18)

0.9999999847700205

In [61]:
def prediction_function(age, affordability):
    weightedSum= coef[0]*age + coef[1]*affordability + intercept
    return sigmoid(weightedSum)

In [62]:
prediction_function(0.19,0)

0.3073327786682918

In [63]:
prediction_function(0.47,1)

0.5890582711586843

In [64]:
def log_loss(y_true, y_predicted): #Log Loss Function
    epsilon=1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [65]:
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [66]:
def gradient_descent(age, affordability, y_true,epochs, loss_threshold): #Gradient Descent From Scratch
    #w1, w2, bias
    w1=w2=1
    bias=0
    rate=0.01
    n=len(age)
    for i in range(epochs):
        weightedSum=w1*age + w2*affordability + bias
        y_predicted= sigmoid_numpy(weightedSum)

        loss=log_loss(y_true,y_predicted)

        weightOneDerivative= (1/n)*np.dot(np.transpose(age), (y_predicted - y_true))
        weightTwoDerivative= (1/n)*np.dot(np.transpose(age), (y_predicted - y_true))

        biasDerivative= np.mean((y_predicted - y_true))

        w1=w1 - rate*weightOneDerivative
        w2=w2 - rate*weightTwoDerivative
        bias=bias - rate*biasDerivative

        print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss <= loss_threshold:
            break

    return w1,w2,bias
    

In [67]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train,4000, 0.5831)

Epoch:0, w1:0.9993973933041767, w2:0.9993973933041767, bias:-0.0028495171524819084, loss:0.7806926831422444
Epoch:1, w1:0.9987970592428416, w2:0.9987970592428416, bias:-0.0056927703182716154, loss:0.779740615424826
Epoch:2, w1:0.9981989963009038, w2:0.9981989963009038, bias:-0.008529764696094837, loss:0.7787930341011233
Epoch:3, w1:0.9976032029351963, w2:0.9976032029351963, bias:-0.011360505556624592, loss:0.7778499248708528
Epoch:4, w1:0.9970096775746142, w2:0.9970096775746142, bias:-0.014184998242070998, loss:0.7769112734058804
Epoch:5, w1:0.9964184186202536, w2:0.9964184186202536, bias:-0.017003248165768738, loss:0.7759770653510021
Epoch:6, w1:0.995829424445552, w2:0.995829424445552, bias:-0.019815260811762273, loss:0.775047286324721
Epoch:7, w1:0.9952426933964298, w2:0.9952426933964298, bias:-0.022621041734388815, loss:0.7741219219200217
Epoch:8, w1:0.994658223791432, w2:0.994658223791432, bias:-0.025420596557859155, loss:0.7732009577051421
Epoch:9, w1:0.9940760139218718, w2:0.9940

(1.1032180517303274, 1.1032180517303274, -1.2567307246789492)

In [68]:
coef, intercept

(array([[2.024238 ],
        [0.6059055]], dtype=float32),
 array([-1.1972238], dtype=float32))

In [69]:
# Creating a Neutral Network from scratch using just Python, no tensorflow or keras.

In [80]:
class myNeuralNetwork:
    def __init__(self):
        self.w1=1
        self.w2=1
        self.bias=0
    
    def fit(self,X,y,epochs, loss_threshold):
        self.w1,self.w2,self.bias = self.gradient_descent(X['age'], X['affordibility'],y, epochs, loss_threshold)

    def predict(self,X_test):
        weightedSum= self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias
        return sigmoid_numpy(weightedSum)

    def gradient_descent(self,age, affordability, y_true,epochs, loss_threshold): 
        #w1, w2, bias
        w1=w2=1
        bias=0
        rate=0.01
        n=len(age)
        for i in range(epochs):
            weightedSum=w1*age + w2*affordability + bias
            y_predicted= sigmoid_numpy(weightedSum)

            loss=log_loss(y_true,y_predicted)

            weightOneDerivative= (1/n)*np.dot(np.transpose(age), (y_predicted - y_true))
            weightTwoDerivative= (1/n)*np.dot(np.transpose(age), (y_predicted - y_true))

            biasDerivative= np.mean((y_predicted - y_true))

            w1=w1 - rate*weightOneDerivative
            w2=w2 - rate*weightTwoDerivative
            bias=bias - rate*biasDerivative

            if i%50==0:
                print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

            if loss <= loss_threshold:
                print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1,w2,bias
        

In [82]:
myModel=myNeuralNetwork()
myModel.fit(X_train_scaled,y_train,epochs=1000, loss_threshold=0.60)
myModel.predict(X_test_scaled)

Epoch:0, w1:0.9993973933041767, w2:0.9993973933041767, bias:-0.0028495171524819084, loss:0.7806926831422444
Epoch:50, w1:0.9721264409433056, w2:0.9721264409433056, bias:-0.13746403534261736, loss:0.7383000339150954
Epoch:100, w1:0.9502583744657668, w2:0.9502583744657668, bias:-0.2572936483980467, loss:0.7053266172717714
Epoch:150, w1:0.9334021890024032, w2:0.9334021890024032, bias:-0.3634883352652293, loss:0.679978129167618
Epoch:200, w1:0.9210904810828004, w2:0.9210904810828004, bias:-0.4573733776624043, loss:0.6606287144160633
Epoch:250, w1:0.9128242736161729, w2:0.9128242736161729, bias:-0.5403322461018617, loss:0.6458948938837756
Epoch:300, w1:0.9081072832560089, w2:0.9081072832560089, bias:-0.6137200106706246, loss:0.6346528855234096
Epoch:350, w1:0.9064688890856926, w2:0.9064688890856926, bias:-0.6788074148635929, loss:0.6260206321548972
Epoch:400, w1:0.9074774133237472, w2:0.9074774133237472, bias:-0.7367502711639708, loss:0.6193229691554992
Epoch:450, w1:0.91074615636774, w2:0.

6     0.390041
25    0.619752
1     0.325049
7     0.401340
2     0.604043
24    0.610804
dtype: float64

In [83]:
coef, intercept

(array([[2.024238 ],
        [0.6059055]], dtype=float32),
 array([-1.1972238], dtype=float32))

In [84]:
model.predict(X_test_scaled)

array([[0.4790391 ],
       [0.6228753 ],
       [0.33377415],
       [0.5043297 ],
       [0.5890583 ],
       [0.6036748 ]], dtype=float32)